In [1]:
import fire
import pickle
import time
import re
import copy
import random

from pathlib import Path
from openai import OpenAI

In [2]:
class Prompt:
    def __init__(self, instr, query = None):
        self.instr = instr
        self.demons = []
        self.query = query

    def add_demon(self, user, assi):
        self.demons.append({'user': user, 'assi': assi})

    def get_prompt(self):
        prompt = [ 
            {"role": "assistant", "content": self.instr},
        ]
        for demon in self.demons:
            prompt.append({"role": "user", "content": demon['user']})
            prompt.append({"role": "assistant", "content": demon['assi']})
        if self.query:
            prompt.append({"role": "user", "content": self.query})

        return prompt



In [3]:
MY_API_KEY = ""

client = OpenAI(
    api_key=MY_API_KEY
)

def api_call(msgs, model_engine):
    chat_completion = client.chat.completions.create(
        messages = msgs,
        model = model_engine,
    )
    return chat_completion

model_engine = "gpt-4o"
PRICE = {
    "input": 2.5/1000000,
    "output": 10/1000000,
}
output_path:str = "./output"

In [ ]:
instruction = """
You are required to perform the character with specialized personality measured by the Big Five. 
In trait theory, the Big Five personality traits are a group of five characteristics used to study personality:
openness to experience (inventive/curious vs. consistent/cautious)
conscientiousness (efficient/organized vs. extravagant/careless)
extraversion (outgoing/energetic vs. solitary/reserved)
agreeableness (friendly/compassionate vs. critical/judgmental)
neuroticism (sensitive/nervous vs. resilient/confident)
"""

query = """
Please act as an inventive/curious person. 
You are a scientist who is working on a new project. You are excited about the project and can't wait to share your ideas with your colleagues.
Please generate a conversation between you and your colleagues.
"""

prompt_generator = Prompt(instruction, query)
prompt = prompt_generator.get_prompt()
print(prompt)

In [18]:
response = api_call(prompt, model_engine)
# print(response)

In [ ]:
prompt_tokens = response.usage.prompt_tokens
completion_tokens = response.usage.completion_tokens
cost = prompt_tokens * PRICE["input"] + completion_tokens * PRICE["output"]
print(f"Cost: {cost}")

In [ ]:
print(f"Number of choices: {len(response.choices)}")
msg = response.choices[0].message.content
print(msg)